In [ ]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4

In [ ]:
import warnings
warnings.filterwarnings('ignore')

: 

In [ ]:
import streamlit as st
import os
COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]
QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

In [ ]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

In [ ]:
from langchain_cohere.llms import Cohere

llm = Cohere()
print(llm.invoke("Come up with a pet name"))

In [ ]:
print(llm.invoke("Summarize the theory of Kahnemann and Tversky"))

In [ ]:
import os
TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

llm = ChatCohere()

internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [internet_search],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

agent_executor.invoke({
    "input": "When has Palantir had its IPO at the NYSE?",
})

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)

class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia"
    )
    
wikipedia_tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [wikipedia_tool],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

response = agent_executor.invoke({
                "input": "Explain the theory of Kahnemann and Tversky",
            })

In [ ]:
wikipedia_tool.run("David Beckham")

In [ ]:
import os
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_cohere import ChatCohere, create_cohere_react_agent

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY



llm = ChatCohere()

prompt = ChatPromptTemplate.from_template("{input}")


tools = load_tools(["openweathermap-api"], llm)
tools.append(wikipedia_tool)


agent = create_cohere_react_agent(
    tools=tools, llm=llm, prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "How is the weather in London?"})

In [ ]:
response["output"]

In [ ]:
! pip install --upgrade --quiet  yfinance

In [ ]:
tools = load_tools(["openweathermap-api"], llm)
internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"
tools.append(internet_search)
tools.append(wikipedia_tool)

In [ ]:
prompt = ChatPromptTemplate.from_template("Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}")

In [ ]:
agent = create_cohere_react_agent(
    tools=tools, llm=llm, prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(agent_executor.invoke({"input": "Who was George Washington?"}))
print(agent_executor.invoke({"input": "What's the weather like in New Orleans?"}))
print(agent_executor.invoke({"input": "Who did win the Indian elections this year?"}))
print(agent_executor.invoke({"input": "How does the Microsoft stock perform?"}))

In [ ]:
print(agent_executor.invoke({"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}))